In [19]:
from policyengine_us import Simulation
import matplotlib.pyplot as plt
import numpy as np
from policyengine_core.reforms import Reform
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from policyengine_core.charts import *

In [6]:
YEAR = 2031
MAX_INCOME = 200000

In [13]:
reform = Reform.from_dict(
    {
        "gov.states.hi.tax.income.deductions.standard.amount.HEAD_OF_HOUSEHOLD": {
            "2024-01-01.2100-12-31": 3212
        },
        "gov.states.hi.tax.income.deductions.standard.amount.JOINT": {
            "2024-01-01.2100-12-31": 4400
        },
        "gov.states.hi.tax.income.deductions.standard.amount.SEPARATE": {
            "2024-01-01.2100-12-31": 2200
        },
        "gov.states.hi.tax.income.deductions.standard.amount.SINGLE": {
            "2024-01-01.2100-12-31": 2200
        },
        "gov.states.hi.tax.income.deductions.standard.amount.SURVIVING_SPOUSE": {
            "2024-01-01.2100-12-31": 4400
        },
    },
    country_id="us",
)

In [7]:
def create_situation(household_type):
    if household_type == "Married with two children":
        situation = {
            "people": {
                "adult1": {"age": {YEAR: 40}},
                "adult2": {"age": {YEAR: 40}},
                "child1": {"age": {YEAR: 10}},
                "child2": {"age": {YEAR: 5}},
            },
            "households": {
                "household": {
                    "members": ["adult1", "adult2", "child1", "child2"],
                    "state_name": {YEAR: "HI"},
                }
            },
            "marital_units": {"marital_unit": {"members": ["adult1", "adult2"]}},
            "tax_units": {
                "tax_unit": {"members": ["adult1", "adult2", "child1", "child2"]}
            },
        }
    elif household_type == "Single parent of two":
        situation = {
            "people": {
                "adult": {"age": {YEAR: 40}},
                "child1": {"age": {YEAR: 10}},
                "child2": {"age": {YEAR: 5}},
            },
            "households": {
                "household": {
                    "members": ["adult", "child1", "child2"],
                    "state_name": {YEAR: "HI"},
                }
            },
            "marital_units": {"marital_unit": {"members": ["adult"]}},
            "tax_units": {"tax_unit": {"members": ["adult", "child1", "child2"]}},
        }
    else:  # Married couple with no kids
        situation = {
            "people": {"adult1": {"age": {YEAR: 40}}, "adult2": {"age": {YEAR: 40}}},
            "households": {
                "household": {
                    "members": ["adult1", "adult2"],
                    "state_name": {YEAR: "HI"},
                }
            },
            "marital_units": {"marital_unit": {"members": ["adult1", "adult2"]}},
            "tax_units": {"tax_unit": {"members": ["adult1", "adult2"]}},
        }

    situation["axes"] = [
        [
            {
                "name": "employment_income",
                "count": 201,
                "min": 0,
                "max": MAX_INCOME,
                "period": YEAR,
            }
        ]
    ]

    return situation

In [8]:
def calculate_income(situation, reform=None):
    simulation = Simulation(situation=situation, reform=reform)
    return simulation.calculate("household_net_income", 2031)

In [21]:
def create_income_comparison_graph():
    household_types = [
        "Married with two children",
        "Single parent of two",
        "Married couple with no kids",
    ]
    fig = make_subplots(
        rows=3, cols=1, subplot_titles=household_types, shared_xaxes=True
    )

    for i, household_type in enumerate(household_types, start=1):
        situation = create_situation(household_type)
        baseline_income = calculate_income(situation)
        reform_income = calculate_income(situation, reform)

        x = np.linspace(0, MAX_INCOME, 201)
        y = baseline_income - reform_income  # Changed to baseline - reform

        fig.add_trace(
            go.Scatter(x=x, y=y, mode="lines", name=household_type), row=i, col=1
        )

        fig.update_yaxes(title_text="Change in Net Income ($)", row=i, col=1)
        fig.update_xaxes(
            title_text="Employment Income ($)", row=i, col=1
        )  # Added x-axis title for each subplot

    fig.update_layout(
        title_text="Impact of Hawaii Tax Reform on Household Net Income (2031)",
        height=900,
        width=800,
        showlegend=False,
    )

    # Add PolicyEngine logo
    fig.add_layout_image(
        dict(
            source="https://policyengine.org/images/logos/policyengine-logo.png",
            xref="paper",
            yref="paper",
            x=1,
            y=-0.1,
            sizex=0.2,
            sizey=0.2,
            xanchor="right",
            yanchor="bottom",
        )
    )

    return fig

In [22]:
# Generate and display the graph
fig = create_income_comparison_graph()
fig = format_fig(fig)
fig.show()